In [41]:
import warnings
from math import comb

import numpy as np
import pandas as pd
from scipy.stats import bernoulli, logistic

warnings.filterwarnings("ignore")

In [42]:
# Pooling====
# GROUP TESTING
# CREATED: Eli P. Fenichel, Yale University
# LAST UPDATED: July 15, 2020
# Probability of a groups testing positive accounting for sensitivity of test
# with prevalence m, group size = group, population = pop


def estimate_pooling_prob_of_pos_groups(group_size, m, pop, se, s_loss):
    shat = se - group_size * s_loss
    p_pos = 1 - (1 - shat * m) ** group_size  # shat es la sensibilidad,
    # p.pos: Probabilidad que un grupo sea positivo
    # (1-prevalencia): Probabilidad que se salga negativo una persona
    # (1-prevalencia)^tamano_grupo: Probabilidad que todos sean negativos en el grupo
    return p_pos

In [43]:
# Group results
def estimate_pooling_tests(group_size, p, pop, se, s_loss, sp):
    p_pos_pool = estimate_pooling_prob_of_pos_groups(group_size, p, pop, se, s_loss)
    p_pos_test = se * p + (1 - sp) * (1 - p)
    # number of groups
    n_group = int(np.round(pop / group_size))
    pos_groups = np.sum(bernoulli.rvs(p_pos_pool, size=n_group))
    neg_groups = n_group - pos_groups  # cambio

    # number of people
    neg_peop_group = neg_groups * group_size
    pos_peop_group = pos_groups * group_size

    pos_peop = pos_peop_group * p_pos_test

    n_groups_pooling = pos_groups + neg_groups
    total_tests_pooling = n_groups_pooling + pos_peop
    test_per_person_pooling = total_tests_pooling / pop

    # the expected number of people in negative groups that are actually
    # false positives with group size = group.size, prevalence p, sensitivity s,
    # in a population p.

    shat = se - group_size * s_loss
    t_prob = (1 - shat) * p + (1 - p)
    expected_number_missed = np.zeros(group_size - 1)
    for x in range(group_size - 1):
        expected_number_missed[x] = (
            comb(group_size, group_size - (x + 1))
            * ((1 - p) / t_prob) ** (group_size - (x + 1))
            * ((1 - shat) * p / t_prob) ** (x + 1)
        )

    exp_m = np.matmul(range(1, group_size), expected_number_missed)
    gm = exp_m * neg_groups

    res = pd.DataFrame(
        {
            "p_pos_pool": [p_pos_pool],
            "p_pos_test": [p_pos_test],
            "pos_groups": [pos_groups],
            "neg_groups": [neg_groups],
            "pos_peop_group": [pos_peop_group],
            "pos_peop": [pos_peop],
            "neg_peop_group": [neg_peop_group],
            "n_groups_pooling": [n_groups_pooling],
            "n_undetected_pos": [gm],
            "undetected_groups": [exp_m],
            "total_tests_pooling": [total_tests_pooling],
            "number_test_per_person_pooling": [test_per_person_pooling],
        }
    )
    return res

In [44]:
def covid_testing_strategies(
    prevalence,
    sens_model,
    spec_model,
    sens_ag_test,
    spec_ag_test,
    sens_pcr_test,
    spec_pcr_test,
    sens_lamp_test,
    spec_lamp_test,
    people,
    cost_pcr,
    cost_antigen,
    cost_lamp,
    group_size,
    strategy,
    budget,
):
    df_out = pd.DataFrame({})
    sens = sens_model
    spec = spec_model
    # for sens in sens_model:
    #     for spec in spec_model:

    # Probability that an a test gives a positive result
    # Antigen
    prob_res_pos_ag = sens_ag_test * prevalence + (1 - spec_ag_test) * (1 - prevalence)
    # LAMP
    prob_res_pos_lamp = sens_lamp_test * prevalence + (1 - spec_lamp_test) * (
        1 - prevalence
    )
    # PCR
    prob_res_pos_pcr = sens_pcr_test * prevalence + (1 - spec_pcr_test) * (
        1 - prevalence
    )
    # Probability that a test gives a negative result
    prob_res_neg_ag = 1 - prob_res_pos_ag
    prob_res_neg_lamp = 1 - prob_res_pos_lamp
    prob_res_neg_pcr = 1 - prob_res_pos_pcr

    # Probability model gives a low risk classification
    # prob_model_low_risk = (1 - sens) * prevalence +
    #   spec * (1 - prevalence)
    # prob_model_high_risk = 1 - prob_model_low_risk

    prob_model_high_risk_ag = sens * prob_res_pos_ag + (1 - spec) * prob_res_neg_ag
    # prob_model_high_risk_ag = 1 - prob_model_low_risk_ag

    prob_model_high_risk_pcr = sens * prob_res_pos_pcr + (1 - spec) * prob_res_neg_pcr
    # prob_model_high_risk_pcr = 1 - prob_model_low_risk_pcr

    q_model_high_risk_ag = logistic.ppf(prob_model_high_risk_ag)
    q_model_high_risk_pcr = logistic.ppf(prob_model_high_risk_pcr)

    prob_model_high_risk = logistic.cdf(
        0.5 * (q_model_high_risk_ag + q_model_high_risk_pcr)
    )
    prob_model_low_risk = 1 - prob_model_high_risk

    # PPV y NPV of model
    npv_model_ag = spec * prob_res_neg_ag / prob_model_low_risk
    npv_model_pcr = spec * prob_res_neg_pcr / prob_model_low_risk

    q_npv_model_low_risk_ag = logistic.ppf(npv_model_ag)
    q_npv_model_low_risk_pcr = logistic.ppf(npv_model_pcr)

    npv_model = logistic.cdf(0.5 * (q_npv_model_low_risk_ag + q_npv_model_low_risk_pcr))
    #
    # npv_model <- spec * (1 - prevalence) /
    #   ((1 - sens) * prevalence + spec * (1 - prevalence))

    # People in risk
    people_high_risk = np.sum(bernoulli.rvs(prob_model_high_risk, size=people))
    people_low_risk = people - people_high_risk

    # Probability that given High risk classification
    # the patient has a positive result with antigen

    prob_res_pos_given_high_risk_ag = sens * prob_res_pos_ag / prob_model_high_risk

    prob_res_neg_given_high_risk_ag = (
        (1 - spec) * prob_res_neg_ag / prob_model_high_risk
    )

    prob_res_pos_given_low_risk_ag = (1 - sens) * prob_res_pos_ag / prob_model_low_risk

    prob_res_pos_given_high_risk_lamp = sens * prob_res_pos_lamp / prob_model_high_risk

    prob_res_neg_given_high_risk_lamp = 1 - prob_res_pos_given_high_risk_lamp

    prob_res_pos_given_high_risk_pcr = sens * prob_res_pos_pcr / prob_model_high_risk

    prob_res_neg_given_high_risk_pcr = 1 - prob_res_pos_given_high_risk_pcr

    # Probability that given High risk classification
    # the patient has a negative result with antigen
    prob_res_neg_given_high_risk_ag = (
        (1 - spec) * prob_res_neg_ag / prob_model_high_risk
    )

    # PPV golden test
    ppv_golden_test = (
        sens_pcr_test
        * prevalence
        / (sens_pcr_test * prevalence + (1 - spec_pcr_test * (1 - prevalence)))
    )

    # Strategy 1 -------------------------------------------------------
    if strategy == 1:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            # Probability of people tested before of 5 days since symptoms onset.
            prob_symp_less_5d = 1 - prob_symp_more_5d

            #             people_low_risk = 0

            #             # Number of antigen tests done
            #             number_alt_tests = np.sum(
            #                 bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            #             )

            #             # Number of PCR tests done
            #             number_pcr_tests = (
            #                 people_high_risk
            #                 - number_alt_tests
            #                 + np.sum(
            #                     bernoulli.rvs(
            #                         prob_res_neg_given_high_risk_ag, size=number_alt_tests
            #                     )
            #                 )
            #             )

            #             number_test_per_person = (number_pcr_tests + number_alt_tests) / (
            #                 people_high_risk
            #             )

            #             number_positive_reported = (
            #                 people_res_pos_given_high_risk_ag
            #                 + people_res_pos_given_res_neg_ag_high_risk
            #                 + people_res_pos_given_high_risk_pcr
            #                 + pooling_tests_low_risk["pos_peop"]
            #             )

            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag, size=people_low_risk)
            )

            number_alt_tests = people_symp_less_5d

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag, size=people_symp_less_5d)
            )
            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            number_pcr_tests = (
                people_high_risk
                - number_alt_tests
                + np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_ag, size=number_alt_tests
                    )
                )
            )
            number_test_per_person = (number_pcr_tests + number_alt_tests) / (
                people_high_risk
            )
            # Total costs.
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
            )

            stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            efficiency = (number_positive_reported / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": [total_cost_alt],
                    "total_cost_pcr": [total_cost_pcr],
                    "total_cost": [total_cost],
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 2:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d

            pooling_tests_full_prevalence = pd.DataFrame()

            pooling_tests_full_prevalence = estimate_pooling_tests(
                group_size=group_size,
                p=prevalence,
                pop=people_low_risk,
                se=sens_pcr_test,
                s_loss=0,
                sp=spec_pcr_test,
            )

            pooling_tests_low_risk = estimate_pooling_tests(
                group_size=group_size,
                p=1 - npv_model,
                pop=people_low_risk,
                se=sens_pcr_test,
                s_loss=0,
                sp=spec_pcr_test,
            )

            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag, size=people_low_risk)
            )

            number_alt_tests = people_symp_less_5d

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag, size=people_symp_less_5d)
            )
            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            number_pcr_tests = (
                people_high_risk
                - number_alt_tests
                + np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_ag, size=number_alt_tests
                    )
                )
                + pooling_tests_low_risk["total_tests_pooling"].values
            )
            number_test_per_person = (number_pcr_tests + number_alt_tests) / (people)
            # Total costs.
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
                + pooling_tests_low_risk["pos_peop"]
            )

            # number_positive_reported <-
            #   people_high_risk * prob_symp_less_5d *
            #   (
            #     prob_res_pos_given_high_risk_ag +
            #       prob_res_neg_given_high_risk_ag *
            #         prob_res_pos_given_high_risk_pcr
            #   ) +
            #   people_high_risk * prob_symp_more_5d *
            #     prob_res_pos_given_high_risk_pcr +
            #   pooling_tests_low_risk$pos_peop
            stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            efficiency = (number_positive_reported / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 3:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d
            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag, size=people_low_risk)
            )

            number_alt_tests = (
                people_symp_less_5d + people_low_risk + people_res_pos_given_low_risk_ag
            )

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag, size=people_symp_less_5d)
            )

            number_pcr_tests = people_symp_more_5d + people_res_neg_given_high_risk_ag

            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            people_confirmed_retest_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag**2, size=people_low_risk)
            )

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
                + people_confirmed_retest_ag
            )

            number_test_per_person = (number_pcr_tests + number_alt_tests) / (people)
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            efficiency = (number_positive_reported / (total_cost)) * (
                (people_high_risk + people_low_risk) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 4:
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d
            prob_model_first_responder_pcr = 0.01
            people_first_responder = np.sum(
                bernoulli.rvs(prob_model_first_responder_pcr, size=people)
            )
            people_high_risk = np.sum(
                bernoulli.rvs(
                    prob_model_high_risk, size=people - people_first_responder
                )
            )
            people_low_risk = people - people_first_responder - people_high_risk

            # Number of test
            number_pcr_tests = people_first_responder

            number_lamp_tests = (
                np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_lamp, size=people_high_risk
                    )
                )
                + people_high_risk
            )

            number_ag_tests = (
                np.sum(
                    bernoulli.rvs(prob_res_pos_given_low_risk_ag, size=people_low_risk)
                )
                + people_low_risk
            )

            number_alt_tests = number_lamp_tests + number_ag_tests

            Highrisk = np.sum(
                bernoulli.rvs(prob_res_pos_given_high_risk_pcr, size=people_high_risk)
            )

            number_positive_reported = (
                np.sum(bernoulli.rvs(prob_res_pos_pcr, size=people_first_responder))
                + Highrisk
                + np.sum(
                    bernoulli.rvs(1 - prob_res_pos_given_high_risk_pcr, size=Highrisk)
                )
                + np.sum(
                    bernoulli.rvs(
                        prob_res_pos_given_low_risk_ag**2, size=people_low_risk
                    )
                )
            )

            number_test_per_person = (number_pcr_tests + number_alt_tests) / (people)

            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost_alt = (
                cost_lamp * number_lamp_tests + cost_antigen * number_ag_tests
            )
            total_cost = total_cost_alt + total_cost_pcr

            stock_capacity = ((number_pcr_tests + number_alt_tests) / total_cost) * (
                (people_high_risk + people_low_risk + people_first_responder) / people
            )

            efficiency = (number_positive_reported / total_cost) * (
                (people_high_risk + people_low_risk + people_first_responder) / people
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [
                        people_high_risk + people_low_risk + people_first_responder
                    ],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "people_first_responder": [people_first_responder],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "sens_lamp_test": [sens_lamp_test],
                    "spec_lamp_test": [spec_lamp_test],
                    "prob_model_first_responder_pcr,": [prob_model_first_responder_pcr],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )
            df_out = pd.concat([df_out, df2])

    return df_out

In [45]:
def r_covid_testing_strategies(
    prevalence,
    sens_model,
    spec_model,
    strategy,
    people,
    n_simul=1000,
    sens_ag_test=0.8,
    spec_ag_test=0.95,
    sens_pcr_test=0.95,
    spec_pcr_test=0.95,
    sens_lamp_test=0.95,
    spec_lamp_test=0.95,
    cost_pcr=100,
    cost_antigen=50,
    cost_lamp=20,
    group_size=5,
    budget=100000,
):
    resultados = pd.DataFrame()
    for s in range(n_simul):
        df = covid_testing_strategies(
            prevalence,
            sens_model,
            spec_model,
            sens_ag_test,
            spec_ag_test,
            sens_pcr_test,
            spec_pcr_test,
            sens_lamp_test,
            spec_lamp_test,
            people,
            cost_pcr,
            cost_antigen,
            cost_lamp,
            group_size,
            strategy,
            budget,
        )
        df["nsimul"] = s + 1
        resultados = pd.concat([resultados, df], ignore_index=True)
    return resultados

In [46]:
model_parameters = pd.read_pickle(filepath_or_buffer="data/model_parameters.pkl")

In [47]:
df_strategy_1 = r_covid_testing_strategies(
    strategy=1,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_1.to_pickle(path="data/df_strategy_1.pkl")
df_strategy_1.head(n=12)

,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,1,0.269096,1000,401,599,0.805556,0.764505,0.8,0.95,0.75,...,238,313,269.096355,1.326683,14700,23800,38500,1381.818182,812.987013,1
1,1,0.269096,1000,401,599,0.805556,0.764505,0.8,0.95,0.50,...,280,287,269.096355,1.224439,10550,28000,38550,1273.670558,744.487678,1
2,1,0.269096,1000,401,599,0.805556,0.764505,0.8,0.95,0.25,...,355,269,269.096355,1.134663,5000,35500,40500,1123.456790,664.197531,1
3,1,0.269096,1000,395,605,0.805556,0.764505,0.8,0.95,0.75,...,257,293,269.096355,1.374684,14300,25700,40000,1357.500000,732.500000,2
4,1,0.269096,1000,395,605,0.805556,0.764505,0.8,0.95,0.50,...,295,259,269.096355,1.205063,9050,29500,38550,1234.760052,671.854734,2
5,1,0.269096,1000,395,605,0.805556,0.764505,0.8,0.95,0.25,...,331,248,269.096355,1.098734,5150,33100,38250,1134.640523,648.366013,2
6,1,0.269096,1000,412,588,0.805556,0.764505,0.8,0.95,0.75,...,268,323,269.096355,1.359223,14600,26800,41400,1352.657005,780.193237,3
7,1,0.269096,1000,412,588,0.805556,0.764505,0.8,0.95,0.50,...,296,297,269.096355,1.254854,11050,29600,40650,1271.832718,730.627306,3
8,1,0.269096,1000,412,588,0.805556,0.764505,0.8,0.95,0.25,...,353,270,269.096355,1.106796,5150,35300,40450,1127.317676,667.490729,3
9,1,0.269096,1000,374,626,0.805556,0.764505,0.8,0.95,0.75,...,231,298,269.096355,1.377005,14200,23100,37300,1380.697051,798.927614,4


In [48]:
df_strategy_2 = r_covid_testing_strategies(
    strategy=2,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_2.to_pickle(path="data/df_strategy_2.pkl")
df_strategy_2.head(n=12)

,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,2,0.269096,1000,369,631,0.805556,0.764505,0.8,0.95,0.75,...,382.773826,322.773826,269.096355,0.656774,13700,38277.382587,51977.382587,1263.576181,620.988995,1
1,2,0.269096,1000,369,631,0.805556,0.764505,0.8,0.95,0.50,...,410.020718,283.020718,269.096355,0.596021,9300,41002.071814,50302.071814,1184.883041,562.642269,1
2,2,0.269096,1000,369,631,0.805556,0.764505,0.8,0.95,0.25,...,474.271754,286.271754,269.096355,0.572272,4900,47427.175405,52327.175405,1093.641592,547.080464,1
3,2,0.269096,1000,388,612,0.805556,0.764505,0.8,0.95,0.75,...,364.642092,325.642092,269.096355,0.665642,15050,36464.209247,51514.209247,1292.152403,632.140330,2
4,2,0.269096,1000,388,612,0.805556,0.764505,0.8,0.95,0.50,...,432.518646,306.518646,269.096355,0.626519,9700,43251.864633,52951.864633,1183.185239,578.862800,2
5,2,0.269096,1000,388,612,0.805556,0.764505,0.8,0.95,0.25,...,490.769682,273.769682,269.096355,0.581770,4550,49076.968224,53626.968224,1084.845371,510.507477,2
6,2,0.269096,1000,379,621,0.805556,0.764505,0.8,0.95,0.75,...,379.395200,335.395200,269.096355,0.675395,14800,37939.520019,52739.520019,1280.624473,635.946630,3
7,2,0.269096,1000,379,621,0.805556,0.764505,0.8,0.95,0.50,...,406.271754,296.271754,269.096355,0.601272,9750,40627.175405,50377.175405,1193.540029,588.107117,3
8,2,0.269096,1000,379,621,0.805556,0.764505,0.8,0.95,0.25,...,471.646236,274.646236,269.096355,0.566646,4750,47164.623610,51914.623610,1091.496378,529.034436,3
9,2,0.269096,1000,365,635,0.805556,0.764505,0.8,0.95,0.75,...,372.271754,309.271754,269.096355,0.631272,12950,37227.175405,50177.175405,1258.085472,616.359433,4


In [49]:
df_strategy_3 = r_covid_testing_strategies(
    strategy=3,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_3.to_pickle(path="data/df_strategy_3.pkl")
df_strategy_3.head(n=12)

,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,3,0.269096,1000,395,605,0.805556,0.764505,0.8,0.95,0.75,...,243,300,269.096355,1.195,47600,24300,71900,1662.030598,417.246175,1
1,3,0.269096,1000,395,605,0.805556,0.764505,0.8,0.95,0.50,...,285,275,269.096355,1.126,42050,28500,70550,1596.031184,389.794472,1
2,3,0.269096,1000,395,605,0.805556,0.764505,0.8,0.95,0.25,...,344,254,269.096355,1.083,36950,34400,71350,1517.869657,355.991591,1
3,3,0.269096,1000,390,610,0.805556,0.764505,0.8,0.95,0.75,...,233,299,269.096355,1.196,48150,23300,71450,1673.897831,418.474458,2
4,3,0.269096,1000,390,610,0.805556,0.764505,0.8,0.95,0.50,...,275,294,269.096355,1.125,42500,27500,70000,1607.142857,420.000000,2
5,3,0.269096,1000,390,610,0.805556,0.764505,0.8,0.95,0.25,...,336,242,269.096355,1.083,37350,33600,70950,1526.427061,341.085271,2
6,3,0.269096,1000,398,602,0.805556,0.764505,0.8,0.95,0.75,...,237,308,269.096355,1.194,47850,23700,71550,1668.763103,430.468204,3
7,3,0.269096,1000,398,602,0.805556,0.764505,0.8,0.95,0.50,...,281,289,269.096355,1.128,42350,28100,70450,1601.135557,410.220014,3
8,3,0.269096,1000,398,602,0.805556,0.764505,0.8,0.95,0.25,...,354,261,269.096355,1.096,37100,35400,72500,1511.724138,360.000000,3
9,3,0.269096,1000,395,605,0.805556,0.764505,0.8,0.95,0.75,...,230,314,269.096355,1.185,47750,23000,70750,1674.911661,443.816254,4


In [50]:
df_strategy_4 = r_covid_testing_strategies(
    strategy=4,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_4.to_pickle(path="data/df_strategy_4.pkl")
df_strategy_4.head(n=12)

,strategy,prevalence,people,people_high_risk,people_low_risk,people_first_responder,sens_model,spec_model,sens_ag_test,spec_ag_test,...,number_pcr_tests,number_positive_reported,number_positive_theoretical,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul
0,4,0.269096,1000,372,616,12,0.805556,0.764505,0.8,0.95,...,12,298,269.096355,1.196,43810,1200,45010,2657.187292,662.075094,1
1,4,0.269096,1000,399,589,12,0.805556,0.764505,0.8,0.95,...,12,341,269.096355,1.201,42710,1200,43910,2735.140059,776.588476,1
2,4,0.269096,1000,371,622,7,0.805556,0.764505,0.8,0.95,...,7,325,269.096355,1.178,43430,700,44130,2669.385905,736.460458,1
3,4,0.269096,1000,372,616,12,0.805556,0.764505,0.8,0.95,...,12,321,269.096355,1.193,43390,1200,44590,2675.487778,719.892353,2
4,4,0.269096,1000,399,590,11,0.805556,0.764505,0.8,0.95,...,11,361,269.096355,1.224,43370,1100,44470,2752.417360,811.783225,2
5,4,0.269096,1000,387,598,15,0.805556,0.764505,0.8,0.95,...,15,340,269.096355,1.224,43800,1500,45300,2701.986755,750.551876,2
6,4,0.269096,1000,406,586,8,0.805556,0.764505,0.8,0.95,...,8,363,269.096355,1.216,43180,800,43980,2764.893133,825.375171,3
7,4,0.269096,1000,382,608,10,0.805556,0.764505,0.8,0.95,...,10,341,269.096355,1.210,43800,1000,44800,2700.892857,761.160714,3
8,4,0.269096,1000,381,614,5,0.805556,0.764505,0.8,0.95,...,5,348,269.096355,1.200,43580,500,44080,2722.323049,789.473684,3
9,4,0.269096,1000,345,651,4,0.805556,0.764505,0.8,0.95,...,4,289,269.096355,1.218,46060,400,46460,2621.609987,622.040465,4
